# 06 Worst-case Report

Rank worst runs by tail latency and export a summary table.


In [1]:
from pathlib import Path
import os
import pandas as pd
import numpy as np

ANALYZE_DIR = Path('../outputFiles/analyze').resolve()
REPORT_PREFIX = os.environ.get('REPORT_PREFIX', 'analysis_reports')
COLLECT_PREFIX = os.environ.get('COLLECT_PREFIX', REPORT_PREFIX)
REPORT_DIR = (ANALYZE_DIR / REPORT_PREFIX)
COLLECT_DIR = (ANALYZE_DIR / COLLECT_PREFIX)
STATS_CSV = None  # set to a specific file path if needed
TOPK_CSV = None   # set to a specific file path if needed

def pick_latest(pattern):
    files = sorted(COLLECT_DIR.glob(pattern))
    if not files:
        raise FileNotFoundError(f'No files matched: {pattern}')
    return files[-1]

stats_path = Path(STATS_CSV) if STATS_CSV else pick_latest('collected_stats_*.csv')
topk_path = Path(TOPK_CSV) if TOPK_CSV else pick_latest('collected_topk_*.csv')

print('stats:', stats_path)
print('topk :', topk_path)

stats_df = pd.read_csv(stats_path)
topk_df = pd.read_csv(topk_path)


stats: /home/gt/research/DiskANN/scripts/paramAnalysis/gridSearch/outputFiles/analyze/siftsmall01/collected_stats_siftsmall01_20260102_190313.csv
topk : /home/gt/research/DiskANN/scripts/paramAnalysis/gridSearch/outputFiles/analyze/siftsmall01/collected_topk_siftsmall01_20260102_190313.csv


In [2]:
out_tables = (REPORT_DIR / 'tables')
out_tables.mkdir(parents=True, exist_ok=True)

cols = [
    'run_prefix','recall_mean','latency_p99_us','latency_p999_us',
    'ios_p99','queue_depth_p99','io_us_p99','cpu_us_p99',
    'thread_util_p99','expanded_revisit_ratio','node_counts_top10_share',
]
cols = [c for c in cols if c in stats_df.columns]
worst = stats_df.sort_values('latency_p99_us', ascending=False).head(20)[cols]
worst.to_csv(out_tables / 'worst_runs.csv', index=False)
worst


,run_prefix,recall_mean,latency_p99_us,latency_p999_us,ios_p99,queue_depth_p99,io_us_p99,cpu_us_p99,thread_util_p99,expanded_revisit_ratio,node_counts_top10_share
2233,S85_siftsmall_R32_L64_B2_M2_W1_L100_K100_cache...,0.9879,52960.5625,52960.5625,102.0,1.0,52606.5117,290.2559,0.9983,0.494719,0.020145
1290,S46_siftsmall_R16_L64_B2_M2_W1_L100_K100_cache...,0.9883,16292.5059,16292.5059,119.0,1.0,16140.3604,135.3971,0.9989,0.500000,0.019895
3131,S83_siftsmall_R64_L256_B2_M2_W1_L100_K100_cach...,0.9996,15847.9062,15847.9062,103.0,1.0,15336.8389,459.0899,0.9977,0.497112,0.019779
11,S110_siftsmall_R128_L256_B2_M2_W2_L100_K100_ca...,1.0000,14151.6768,14151.6768,103.0,2.0,13589.1562,678.9238,0.9960,0.499122,0.019700
176,S25_siftsmall_R128_L256_B2_M2_W1_L100_K100_cac...,1.0000,13705.6006,13705.6006,101.0,1.0,13079.1455,733.0988,0.9975,0.496165,0.017994
884,S86_siftsmall_R16_L32_B2_M2_W1_L100_K100_cache...,0.9855,13637.0938,13637.0938,116.0,1.0,13460.5273,149.5810,0.9979,0.499373,0.018895
1302,S57_siftsmall_R16_L64_B2_M2_W1_L100_K100_cache...,0.9883,13347.4307,13347.4307,116.0,1.0,13170.6543,156.4850,0.9976,0.500000,0.019895
1334,S86_siftsmall_R16_L64_B2_M2_W1_L100_K100_cache...,0.9883,12854.1982,12854.1982,116.0,1.0,12662.2773,147.3130,0.9984,0.500000,0.019895
1325,S78_siftsmall_R16_L64_B2_M2_W1_L100_K100_cache...,0.9883,11925.0283,11925.0283,119.0,1.0,11716.4658,163.0310,0.9974,0.500000,0.019895
946,S142_siftsmall_R16_L64_B2_M2_W2_L100_K100_cach...,0.9881,11772.7334,11772.7334,105.0,2.0,11626.1123,135.8300,0.9978,0.507548,0.024842
